In [1]:
using OffsetArrays, Plots

On a single instance:
\begin{align}
u =& \min\{h_1,\dots,h_k\} \\
h =& s + \sum_{a=1}^d u_a
\end{align}

Ensemble calculations: $q(h)$ and $p(u)$
\begin{align}
p(u) =& \left[\sum_{h\ge u} q(h)\right]^k - \left[\sum_{h> u} q(h)\right]^k\\
h =& s + \sum_{a=1}^d u_a
\end{align}

In [2]:
prob(v) = OffsetArray(v, 0:length(v)-1)

prob (generic function with 1 method)

In [21]:
function conv(f1, f2; N=length(f 1)+length(f2)-1)
    @assert firstindex(f1)==0 && firstindex(f2)==0
    N1,N2 = length(f1), length(f2)
    f3 = OffsetArray(zeros(eltype(f1),N+1),0:N)
    for x1 in 0:min(N,N1-1), x2 in 0:min(N,N2-1)
        x1+x2>N && continue
        f3[x1+x2] += f1[x1]*f2[x2]
    end
    f3
end

conv (generic function with 2 methods)

In [23]:
f1 = prob([0.0,1.0])
f2 = prob([0.3,0.4,0.1,0.2])
conv(f1,f2)

6-element OffsetArray(::Vector{Float64}, 0:5) with eltype Float64 with indices 0:5:
 0.0
 0.3
 0.4
 0.1
 0.2
 0.0

In [4]:
function poiss(lambda::Real; trunc=10) 
    p = prob([lambda^k * exp(-lambda) / factorial(k) for k in 0:trunc])
    p ./= sum(p)
end

poiss (generic function with 1 method)

\begin{align}
q(n) =& \sum_{l=0}^\infty p_l(l) \sum_{n_1,n_2,\dots,n_l}\delta\left(n, 1+\sum_{a=1}^ln_a\right)\prod_{a=1}^l\hat{q}(n_a)\\
=& \sum_{l=0}^\infty p_l(l) \left(\underbrace{\hat{q}\circledast\dots\circledast\hat{q}}_{l\text{  times}}\circledast\delta_1\right)(n)\\
\end{align}
\begin{equation}
Q(n) = \sum_{m\ge m} q(m)
\end{equation}

In [5]:
function updateQ!(q, Q, qhat, Qhat, pl)
    qnew = zero(q)
    for l in eachindex(pl)
        qnew = pl[l]*prob([0.0,0.1])
        for _ in 1:l
           qnew = conv(qnew, qhat) 
        end
        q .+= qnew
    end
    Q .= cumsum(q)
    nothing
end

updateQ! (generic function with 1 method)

In [6]:
const p=3
alpha = 0.5
pl = poiss(p*alpha)


11-element OffsetArray(::Vector{Float64}, 0:10) with eltype Float64 with indices 0:10:
 0.22313028326128562
 0.33469542489192844
 0.2510215686689463
 0.12551078433447316
 0.047066544125427434
 0.014119963237628232
 0.003529990809407058
 0.000756426602015798
 0.00014182998787796213
 2.3638331312993692e-5
 3.5457496969490535e-6

\begin{equation}
\hat{Q}(n) = \left(Q(n)\right)^{p-1}
\end{equation}
\begin{equation}
\hat{Q}(n) = \sum_{m\ge m} \hat{q}(m)
\end{equation}

In [7]:
function updateQhat!(qhat, Qhat, q, Q, p)
    Qhat .= Q.^p
    qhat .= diff([0; Qhat])  # inverse of cumsum
    nothing
end

updateQhat! (generic function with 1 method)